In [1]:
import os
import pandas as pd
import numpy as np
from pycaret.regression import *
#from multiprocessing import Pool
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from operator import attrgetter

In [2]:
root="/home/sadagopan/amex-default-prediction/"
headers=pd.read_csv(root+"headers.csv")
headers = headers.columns
scols = [col for col in headers if col[0] == 'S']
dcols = [col for col in headers if col[0] == 'D']
rcols = [col for col in headers if col[0] == 'R']
bcols = [col for col in headers if col[0] == 'B']
pcols = [col for col in headers if col[0] == 'P']
labels =pd.read_csv(root+"train_labels.csv")

In [3]:
trains = []
for r, d, f in os.walk(root):
    f.sort()
    for file in f:
#        if "traina" in file or "trainb" in file:
        if 'pkl' not in file and 'csv' not in file and ("xa" in file or "xb" in file or file.startswith("xc")):
            trains.append(os.path.join(root, file))

dates=pd.DataFrame(pd.date_range('2018-04-01','2019-04-30', freq='MS').tolist(), columns=['S_2_dt'])
dates['yearmonth'] = dates['S_2_dt'].dt.strftime("%Y%m")

In [4]:
#traindf = pd.concat(traindfs, ignore_index=True)
headers=pd.read_csv(root+"headers.csv")
headers = headers.columns

traindf = pd.read_csv(root+"/tdata")
tdf = traindf.groupby('customer_ID')['S_2'].agg(['min', 'max', 'count']).reset_index()
#tdf = tdf.merge(labels, on="customer_ID", how="inner")
#for i in range(13):
#    print(i, tdf[tdf['S_2']==i].count())


In [5]:
tdf.rename(columns={"min": "mindate", "max": "maxdate"}, inplace=True)

tdf['maxdate'] = pd.to_datetime(tdf['maxdate'])
tdf['mindate'] = pd.to_datetime(tdf['mindate'])

#tdf['diffcount'] = tdf.apply(lambda x: ((x['maxdate'] - x['mindate'])/np.timedelta64(1, 'M')))

tdf['mondiff'] = (tdf.maxdate.dt.strftime("%Y%m").astype(int) - tdf.mindate.dt.strftime("%Y%m").astype(int) - 87)
tdf['mondiff'] = tdf['mondiff'].apply(lambda x: x+88 if x< 0 else x)
#tdf[tdf['count'] == tdf['mondiff']]

In [ ]:
traindfs = []

for k in range(12):
    traindfs.append(pd.DataFrame(columns=headers))
k=0
for i in range(len(trains)):
    print(trains[i])
    traindf = pd.read_csv(trains[i], names=headers, header=None)
    for j in range(1, 13):
        mask = traindf.customer_ID.isin(tdf[tdf['mondiff'] == j].customer_ID)
        traindfs[j-1] = pd.concat([traindfs[j-1], traindf[mask]], ignore_index=True)
        if traindfs[j-1]['customer_ID'].count() >= 260000:
            traindfs[j-1] = traindfs[j-1].merge(labels, on="customer_ID", how="inner")
            traindfs[j-1].iloc[:260000,].to_csv("x" + str(j-1) + "-" + str(k) + ".csv", index=False)
            traindfs[j-1] = traindfs[j-1].iloc[260000:,]
            traindfs[j-1].drop(columns=['target'], inplace=True)
            k+=1            

for i in range(len(traindfs)):
    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")
    traindfs[i].to_csv("x" + str(i) + ".csv", index=False)

#for i in range(12, len(traindfs)):
#    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")
#    traindfs[i].to_csv("x" + str(i) + "-" + str(k) + ".csv", index=False)


In [ ]:
traindfs = []
i=0
for j in range(12):
    print("x" + str(j) + ".csv")
    traindfs.append(pd.read_csv("x" + str(j) + ".csv"))

    mindate = datetime.strptime('Apr 1 2018 1:33PM', '%b %d %Y %I:%M%p')
    maxdate = mindate + pd.offsets.DateOffset(months=12)
    dates=pd.DataFrame(pd.date_range(mindate, maxdate, freq='MS').tolist(), columns=['S_2_dt'])
    dates['yearmonth'] = dates['S_2_dt'].dt.strftime("%Y%m")
    dates.reset_index(inplace=True)
    dates.rename(columns={'index': 'seq'}, inplace=True)
    dates['seq']=(dates['seq']+1)

    traindfs[i]['S_2'] = pd.to_datetime(traindfs[i]['S_2'])
    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(years=1)
    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(months=1)
    traindfs[i]['yearmonth'] = traindfs[i]['S_2'].dt.strftime("%Y%m")
    traindfs[i]['seq'] = traindfs[i].groupby(['customer_ID']).cumcount().add(1)
    traindfs[i][['sncnt', 'dncnt', 'rncnt', 'bncnt', 'pncnt']] = traindfs[i].apply(lambda x: (sum([(2^scols.index(col)) for col in scols if pd.isnull(x[col])]),
                sum([(2^dcols.index(col)) for col in dcols if pd.isnull(x[col])]),
                sum([(2^rcols.index(col)) for col in rcols if pd.isnull(x[col])]),
                sum([(2^bcols.index(col)) for col in bcols if pd.isnull(x[col])]),
                sum([(2^pcols.index(col)) for col in pcols if pd.isnull(x[col])])), axis=1, result_type='expand')

    dateshift = pd.DataFrame(traindfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'count'])).rename(columns={"min": "S_2"}).reset_index()
    dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
    dateshift['mondiff'] = dateshift['S_2'].dt.month - 4
    dateshift['mondiff'] = dateshift['mondiff'].apply(lambda x: 12+x if x <= 0 else x)

    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)

    traindfs[i]=pd.merge(traindfs[i], pd.merge(pd.DataFrame(traindfs[i]['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), left_on=['customer_ID', 'seq'], right_on=['customer_ID', 'seq'], how='outer')
#    traindfs[i]['yearmonth'] = traindfs[i].apply(lambda x: x['yearmonth_y'] if pd.isnull(x['yearmonth_x']) else x['yearmonth['S_2_'], axis=1)
    traindfs[i]['origmissrcnt'] = traindfs[i].apply(lambda x: (2**((((x['S_2_dt'].year-2018) * 12 ) + x['S_2_dt'].month) - 4)) if (pd.isnull(x['S_2']) and x['seq'] <=j) else 0, axis=1)
    traindfs[i]['totmissrcnt'] = traindfs[i].apply(lambda x: (2**((((x['S_2_dt'].year-2018) * 12 ) + x['S_2_dt'].month) - 4)) if (pd.isnull(x['S_2'])) else 0, axis=1)
    dateshift = pd.merge(dateshift, pd.DataFrame(traindfs[i].groupby(['customer_ID'])['origmissrcnt'].sum()).reset_index(), on ='customer_ID', how='inner')
    dateshift = pd.merge(dateshift, pd.DataFrame(traindfs[i].groupby(['customer_ID'])['totmissrcnt'].sum()).reset_index(), on ='customer_ID', how='inner')
    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2_dt'] if (pd.isnull(x['S_2']) or x['S_2'].month != x['S_2_dt'].month) else x['S_2'], axis=1)
    traindfs[i].drop(columns=['S_2_dt', 'origmissrcnt', 'totmissrcnt', 'seq'], inplace=True)
    traindfs[i] = traindfs[i].rename(columns={"yearmonth_x": "yearmonth"})

    traindfs[i]['mondiff'].fillna(0, inplace=True)
    traindfs[i]['day_of_month'] = traindfs[i]['S_2'].dt.strftime("%d")
    traindfs[i]['day_of_week'] = traindfs[i]['S_2'].dt.strftime("%u")
    traindfs[i]['day_of_year'] = traindfs[i]['S_2'].dt.strftime("%j")
    traindfs[i]['week_of_year'] = traindfs[i]['S_2'].dt.isocalendar().week
    traindfs[i].drop(columns=['mondiff'], inplace=True)
    traindfs[i].drop(columns=['S_2'], inplace=True)
    traindfs[i].drop(columns=['target'], inplace=True)

#    df = traindfs[i].groupby("day_of_year").transform(lambda x: x.fillna(x.mean()))
#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col] = df[col]
    
#    df = traindfs[i].groupby("week_of_year").transform(lambda x: x.fillna(x.mean()))
#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col] = df[col]

#    df = traindfs[i].groupby("day_of_month").transform(lambda x: x.fillna(x.mean()))
#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col] = df[col]

#    df = traindfs[i].groupby("yearmonth").transform(lambda x: x.fillna(x.mean()))
#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col] = df[col]

#    df = traindfs[i].groupby("day_of_week").transform(lambda x: x.fillna(x.mean()))
#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col] = df[col]

    traindfs[i]=traindfs[i].set_index(['customer_ID', 'yearmonth_y']).unstack(1).sort_index(axis=1, level=1)
    traindfs[i] = traindfs[i].reset_index()
    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'origmissrcnt', 'totmissrcnt']], on=['customer_ID'], how='inner')

    cols=traindfs[i].columns
    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
        traindfs[i][col].fillna(traindfs[i][[column for column in cols if col[:len(col)-3] == column[:len(column)-3]]].mean(axis=1), inplace=True)

    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
        traindfs[i][col].fillna(traindfs[i][col].mean(), inplace=True)

    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")
    traindfs[i].drop(columns=[traindfs[i].columns.to_list()[1]], inplace=True)

    coldict={}
    for col in traindfs[i].columns:
        coldict[col] = ''.join(col)
    traindfs[i].rename(coldict, axis=1, inplace=True)

#    for column in [col for col in traindfs[0].columns if col[len(col)-6:] in ['201904', '201903']]:
#        traindfs[0].drop(columns=[column], inplace=True)

    traindfs[i].to_csv("x" + str(j) +"dtcnt.csv", index=False)
    i+=1

traindfs[0]

In [ ]:
traindfs = []
i=0
j=12
for k in range(19):
    print("x" + str(j) + "-" + str(k) + ".csv")
    traindfs.append(pd.read_csv("x" + str(j) + "-" + str(k) + ".csv"))

    mindate = datetime.strptime('Apr 1 2018  1:33PM', '%b %d %Y %I:%M%p')
    maxdate = mindate + pd.offsets.DateOffset(months=j)
    dates=pd.DataFrame(pd.date_range(mindate, maxdate, freq='MS').tolist(), columns=['S_2_dt'])
    dates['yearmonth'] = dates['S_2_dt'].dt.strftime("%Y%m")
    dates.reset_index(inplace=True)
    dates.rename(columns={'index': 'seq'}, inplace=True)
    dates['seq']=dates['seq']+1

    traindfs[i]['S_2'] = pd.to_datetime(traindfs[i]['S_2'])
    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(years=1)
    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(months=1)
    traindfs[i]['yearmonth'] = traindfs[i]['S_2'].dt.strftime("%Y%m")
    traindfs[i]['seq'] = traindfs[i].groupby(['customer_ID']).cumcount().add(1)
    traindfs[i][['sncnt', 'dncnt', 'rncnt', 'bncnt', 'pncnt']] = traindfs[i].apply(lambda x: (sum([(2^scols.index(col)) for col in scols if pd.isnull(x[col])]),
                         sum([(2^dcols.index(col)) for col in dcols if pd.isnull(x[col])]),
                         sum([(2^rcols.index(col)) for col in rcols if pd.isnull(x[col])]),
                         sum([(2^bcols.index(col)) for col in bcols if pd.isnull(x[col])]),
                         sum([(2^pcols.index(col)) for col in pcols if pd.isnull(x[col])])), axis=1, result_type='expand')

    dateshift = pd.DataFrame(traindfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'count'])).rename(columns={"min": "S_2"}).reset_index()
    dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
    dateshift['mondiff'] = dateshift['S_2'].dt.month - 4
    dateshift['mondiff'] = dateshift['mondiff'].apply(lambda x: 12+x if x <0 else x)

    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
    traindfs[i]['yearmonthnew'] = traindfs[i]['S_2'].dt.strftime("%Y%m")

    traindfs[i]=pd.merge(traindfs[i], pd.merge(pd.DataFrame(traindfs[i]['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'seq'], how='inner')
    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2_dt'] if pd.isnull(x['S_2']) else x['S_2'], axis=1)
    traindfs[i].drop(columns=['S_2_dt', 'yearmonth_y'], inplace=True)
    traindfs[i] = traindfs[i].rename(columns={"yearmonth_x": "yearmonth"})

    traindfs[i]['mondiff'].fillna(0, inplace=True)
    traindfs[i]['day_of_month'] = traindfs[i]['S_2'].dt.strftime("%d")
    traindfs[i]['day_of_week'] = traindfs[i]['S_2'].dt.strftime("%u")
    traindfs[i]['day_of_year'] = traindfs[i]['S_2'].dt.strftime("%j")
    traindfs[i].drop(columns=['mondiff'], inplace=True)
    traindfs[i].drop(columns=['S_2'], inplace=True)
    traindfs[i].drop(columns=['target'], inplace=True)

    traindfs[i]=traindfs[i].set_index(['customer_ID', 'yearmonthnew']).unstack(1).sort_index(axis=1, level=1)
    traindfs[i] = traindfs[i].reset_index()
    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'count']], on=['customer_ID'], how='inner')

    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
        traindfs[i][col].fillna(traindfs[i][[column for column in traindfs[i].columns.tolist() if col[:len(col)-3] == column[:len(column)-3]]].mean(axis=1), inplace=True)

    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
        traindfs[i][col].fillna(traindfs[i][col].mean(), inplace=True)

    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")
    traindfs[i].drop(columns=[traindfs[i].columns.to_list()[1]], inplace=True)

    coldict={}
    for col in traindfs[i].columns:
        coldict[col] = ''.join(col)
    traindfs[i].rename(coldict, axis=1, inplace=True)

    for column in [col for col in traindfs[0].columns if col[len(col)-6:] in ['201904', '201903']]:
        traindfs[0].drop(columns=[column], inplace=True)

    traindfs[i].to_csv("x" + str(j) + "-" + str(k) +"dtcnt.csv", index=False)
    i+=1

traindfs[0]

In [ ]:
traindfs = []

for j in range(len(trains)):
    i = j#-18
    print(trains[j])
    traindfs.append(pd.read_csv(trains[j], names=headers, header=None))

    traindfs[i]['S_2'] = pd.to_datetime(traindfs[i]['S_2'])
    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(years=1)
    traindfs[i]['S_2'] = traindfs[i]['S_2'] + pd.offsets.DateOffset(months=1)
    traindfs[i]['yearmonth'] = traindfs[i]['S_2'].dt.strftime("%Y%m")

    dateshift = pd.DataFrame(traindfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'count'])).rename(columns={"min": "S_2"}).reset_index()
    dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
    dateshift['mondiff'] = dateshift['S_2'].dt.month - 4
    dateshift['mondiff'] = dateshift['mondiff'].apply(lambda x: 12+x if x <0 else x)

    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)

    traindfs[i]=pd.merge(traindfs[i], pd.merge(pd.DataFrame(traindfs[i]['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
    traindfs[i]['S_2'] = traindfs[i].apply(lambda x: x['S_2_dt'] if pd.isnull(x['S_2']) else x['S_2'], axis=1)
    traindfs[i].drop(columns=['S_2_dt'], inplace=True)

    traindfs[i]['mondiff'].fillna(0, inplace=True)
#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col] = traindfs[i].groupby('customer_ID')[col].transform(lambda grp: grp.fillna(np.mean(grp)))
#    for col in traindfs[i].columns[traindfs[i].isna().any()]:
#        if traindfs[i][col].dtype in ['float64', 'int64']:
#            traindfs[i][col].fillna(traindfs[i][col].mean(), inplace=True)
    traindfs[i]['day_of_month'] = traindfs[i]['S_2'].dt.strftime("%d")
    traindfs[i]['day_of_week'] = traindfs[i]['S_2'].dt.strftime("%u")
    traindfs[i]['day_of_year'] = traindfs[i]['S_2'].dt.strftime("%j")
    traindfs[i].drop(columns=['mondiff'], inplace=True)
    traindfs[i].drop(columns=['S_2'], inplace=True)

    traindfs[i]=traindfs[i].set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1)
    traindfs[i] = traindfs[i].reset_index()
    traindfs[i] = pd.merge(traindfs[i], dateshift[['customer_ID', 'count']], on=['customer_ID'], how='inner')

#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col].fillna(traindfs[i][[column for column in traindfs[i].columns.tolist() if col[:len(col)-3] == column[:len(column)-3]]].mean(axis=1), inplace=True)

#    for col in traindfs[i][traindfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        traindfs[i][col].fillna(traindfs[i][col].mean(), inplace=True)

    traindfs[i] = traindfs[i].merge(labels, on="customer_ID", how="inner")
    traindfs[i].drop(columns=[traindfs[i].columns.to_list()[1]], inplace=True)

    coldict={}
    for col in traindfs[i].columns:
        coldict[col] = ''.join(col)
    traindfs[i].rename(coldict, axis=1, inplace=True)

    traindfs[i].to_csv(trains[i]+"nona.csv", index=False, header=False)

traindfs[0][traindfs[0]['customer_ID']==0].to_csv(root+"/trains/headers.csv", index=False)
traindfs[0]

In [ ]:
#traindf = pd.concat(traindfs, ignore_index=True)
headers=pd.read_csv(root+"headers.csv")
headers = headers.columns

traindf = pd.read_csv(root+"/tdata")


In [ ]:
#traindf = traindf[traindf['target']==1]
#for col in [col for col in traindf.columns if (col[len(col)-6:]=='201804')]:
#    traindf = traindf[~traindf[col].isnull()]

#traindf=traindf[~traindf['B_12201804'].isnull()]
#traindf['B_42201804'].unique()

#(traindf.groupby('customer_ID')['S_2'].agg(["min", "max"]).reset_index())['min'].max(), (traindf.groupby('customer_ID')['S_2'].agg(["min", "max"]).reset_index())['max'].min()

tdf = traindf.groupby('customer_ID')['S_2'].count().reset_index()
tdf = tdf.merge(labels, on="customer_ID", how="inner")
for i in range(1, 13):
    print(i, tdf[tdf['S_2']==i].count())

#traindf.groupby('customer_ID')['S_2'].count().reset_index()

In [ ]:
headers=pd.read_csv(root+"/trains/headers.csv")
headers = headers.columns

traindf=pd.DataFrame()

for i in range(12):
    traindf = pd.concat([traindf, pd.read_csv(trains[i]+"new.csv", names=headers, header=None)], ignore_index=True)
print("loaded data...")

for col in [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)]:
    traindf[col] = traindf[col].astype(str)
    traindf[col] = traindf[col].astype('category')
print("updated categories...")

traindf.groupby(['target'])['target'].count()


In [ ]:
s = setup(data=traindf, target='target',
          transformation=True,
          ignore_features=['customer_ID'],
          categorical_features = [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)],
          numeric_features=[col for col in traindf.columns if ('day_of_month') in col or ('day_of_year' in col)],
#          normalize = True,
          remove_outliers=True, handle_unknown_categorical=True,fix_imbalance=True,
          combine_rare_levels=True, create_clusters=True,
          feature_selection=True, remove_multicollinearity=True, pca=True, ignore_low_variance=True,
          train_size=0.8, fold=6,
          use_gpu=True,
          silent = True, session_id = 123, verbose=True)
save_config("/amex/dt3regconfig.pkl")

In [ ]:
%matplotlib inline
#eda()

In [ ]:
#load_config("/amex/dtclassconfig.pkl")
dt = create_model('dt', fold=6, verbose=True)
print("created")
save_model(dt, "/amex/models/dt2reg")

In [ ]:
save_model(dt, "/amex/models/dt1Lclass")

In [ ]:
dt = tune_model(dt, fold=6, n_iter=10, choose_better=True)
print("tuned")
save_model(dt, "e:/rebid/amex/models/dt1Lclasstuned")

In [ ]:
dt = finalize_model(dt)
print("finalized")
#optimize_threshold(dt)
#print("optimized")
save_model(dt, "e:/rebid/amex/models/dt1Lclassfin")

In [ ]:
calibrated_dt = calibrate_model(dt)
save_model(calibrated_dt, "/amex/models/dt1Lclasscalib")

In [ ]:
plot_model(dt, plot='calibration')
plot_model(model, plot='calibration')

In [ ]:
dtmodels=[]
dtsets=[]
headers=pd.read_csv(root+"/trains/headers.csv")
headers = headers.columns
n=6
algos=['dt', 'catboost', 'lightgbm']
loadconfig=True
loadmodels=[False, False, False]

for i in range(1):
    if loadconfig:
        dtsets.append(load_config(root+"/configs/" + 'dt' + str(n) + "-" + str(i) + "config.pkl"))
    else:
        print(i, end="")
        traindf=pd.DataFrame()
        for j in range(i*n, (i*n)+n):
            print("."+str(j), end="")
            traindf = pd.concat([traindf, pd.read_csv(trains[i]+"new1.csv", names=headers, header=None)], ignore_index=True)

        for col in [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)]:
            traindf[col] = traindf[col].astype(str)
            traindf[col] = traindf[col].astype('category')

        dtsets.append(setup(data=traindf, target='target',
              transformation=True,
              ignore_features=['customer_ID'],
              categorical_features = [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)],
              numeric_features=[col for col in traindf.columns if ('day_of_month') in col or ('day_of_year' in col)],
              remove_outliers=True, handle_unknown_categorical=True,
              combine_rare_levels=True, 
              create_clusters=True,
#              transform_target=True, transform_target_method='yeo-johnson',
              polynomial_features=True,
              feature_selection=True, remove_multicollinearity=True, pca=True, ignore_low_variance=True,
              train_size=0.8, fold=6,
              use_gpu=True,
              silent = True, session_id = 123, verbose=True))
        save_config(root+"/configs/" + algo + str(n) + "-" + str(i) + "config.pkl")

    for j in range(len(algos)):
        if loadmodels[j]:
            dtmodels.append(load_model(root+"/models/" + algos[j] + str(n)+"-"+str(i)+"reg"))
        else:
            dtmodels.append(create_model(algos[j], fold=6, verbose=True))
            save_model(dtmodels[i], root+"/models/" + algos[j] + str(n)+"-"+str(i)+"reg")

dtmodels.append(stack_models(dtmodels, meta_model = dtmodels[len(dtmodels)-1]))
save_model(dtmodels[len(dtmodels)-1], root+"/models/" + "".join(algos) + "stackedreg")

In [4]:
algos=['dt', 'catboost']
loadconfig=False
loadmodels=[False, False, False, False]

for i in range(1):
    dtmodels=[]
    if loadconfig:
        load_config(root+"/configs/" + "x" + str(i) + "config.pkl")
    else:
#        print("x" + str(i) + "dtcnt.csv", end="")
#        traindf = pd.read_csv("x" + str(i) + "dtcnt.csv")
        print("xall.csv", end="")
        traindf = pd.read_csv("xall.csv")

        for col in [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)]:
            traindf[col] = traindf[col].astype(str)
            traindf[col] = traindf[col].astype('category')

        s = setup(data=traindf, target='target',
              transformation=True,
              normalize=True,
              ignore_features=['customer_ID'],
              categorical_features = [col for col in traindf.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)],
              numeric_features=[col for col in traindf.columns if ('day_of_month') in col or ('day_of_year' in col)],
#              remove_outliers=True, handle_unknown_categorical=True,
#              combine_rare_levels=True, 
#              create_clusters=True,
              transform_target=True, transform_target_method='yeo-johnson',
              trigonometry_features=True,
              polynomial_features=True, #polynomial_degree=4,
              feature_selection=True,# remove_multicollinearity=True, pca=True, ignore_low_variance=True,
              train_size=0.9, fold=10,
              use_gpu=True,
              silent = True, session_id = 123, verbose=True)
        save_config(root+"/configs/" + "x" + str(i) + "config.pkl")

    for j in range(len(algos)):
        if loadmodels[j]:
            dtmodels.append(load_model(root+"/models/" + algos[j] + "-x"+str(i)+"reg"))
        else:
            try:
                dtmodels.append(create_model(algos[j], fold=10, verbose=True))
                save_model(dtmodels[j], root+"/models/" + algos[j] + "-x"+str(i)+"reg")
#                try:
#                    dtmodels[j] = tune_model(dtmodels[j], fold=10, n_iter=15, verbose=True)
#                    save_model(dtmodels[j], root+"/models/" + algos[j] + "-x"+str(i)+"reg")
#                except:
#                    print("Unable to tune for: ", algos[j])
            except:
                print("Unable to build model for:", algos[j])


    stacked=stack_models(dtmodels, meta_model=dtmodels[len(dtmodels)-1])
    save_model(stacked, root+"/models/" + "x" + str(i) + "stackedreg")
#    try:
#        stacked = tune_model(stacked, fold=10, n_iter=15, verbose=True)
#        save_model(stacked, root+"/models/" + "x" + str(i) + "stackedreg")
#    except:
#        print("Unable to tune for stacking")
    dtmodels.append(stacked)


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,0.2735,0.2735,0.5230,-0.1149,0.3625,0.3152
1,0.2598,0.2598,0.5097,-0.0601,0.3533,0.3005
2,0.2690,0.2690,0.5186,-0.1026,0.3595,0.3161
3,0.2732,0.2732,0.5227,-0.1211,0.3623,0.3319
4,0.2639,0.2639,0.5137,-0.0782,0.3561,0.3001
5,0.2697,0.2697,0.5193,-0.1082,0.3600,0.3159
6,0.2657,0.2657,0.5154,-0.0833,0.3573,0.3088
7,0.2669,0.2669,0.5166,-0.0876,0.3581,0.3121
8,0.2676,0.2676,0.5173,-0.1002,0.3586,0.3266


Transformation Pipeline and Model Successfully Saved


In [ ]:
dtmodels=[]
dtsets=[]
headers=pd.read_csv(root+"/trains/headers.csv")
headers = headers.columns
n=18
algos=['dt', 'catboost', 'lightgbm']

for i in range(1):
#    for algo in algos:
#        if algo == "catboost":
#            dtmodels.append(load_model(root+"/models/" + algo + str(n)+"-"+str(i)+"reg"))
#    dtmodels.append(load_model(root+"/models/" + "".join(algos) + str(n) + "-" + str(i) + "stackedreg"))
    dtmodels.append(load_model(root+"/models/" + "x" + str(i) + "stackedreg"))

In [ ]:
stacked=stack_models(dtmodels, meta_model=dtmodels[0])
#stacked = tune_model(stacked, fold=10, n_iter=15, verbose=True)
dtmodels.append(stacked)
save_model(stacked, root+"/models/" + "x" + str(i) + "stackedreg")

In [ ]:
tests = []
for r, d, f in os.walk(root+"test"):
#    print(r)
#    if r ==root+'/test':
        f.sort()
        for file in f:
            if '.idx' not in file and '.csv' not in file and 'cust' not in file and 'preds' not in file and ('testa' in file or 'testb' in file or 'testc' in file or 'testd' in file or 'teste' in file or 'testf' in file):
                tests.append(os.path.join(r, file))
len(tests), tests[len(tests)-1]

(152, '/home/sadagopan/amex-default-prediction/test/testfv')

In [ ]:
#traindf = pd.concat(traindfs, ignore_index=True)
headers=pd.read_csv(root+"headers.csv")
headers = headers.columns

testdf = pd.read_csv(root+"/tstdata")
tdf = testdf.groupby('customer_ID')['S_2'].count().reset_index()
for i in range(1, 13):
    print(i, tdf[tdf['S_2']==i].count())


In [44]:
print(tdf[tdf['S_2']==2].count())

customer_ID    811329
S_2            811329
dtype: int64


In [ ]:
testdfs = []

for k in range(12):
    testdfs.append(pd.DataFrame(columns=headers))

for i in range(len(tests)):
    print(tests[i])
    testdf = pd.read_csv(tests[i], names=headers, header=None)
    for j in range(1, 13):
        mask = testdf.customer_ID.isin(tdf[tdf['S_2']==j].customer_ID)
        testdfs[j-1] = pd.concat([testdfs[j-1], testdf[mask]])

for i in range(len(testdfs)):
    testdfs[i].to_csv("t" + str(i) + ".csv", index=False)


In [ ]:
testdfs = []

for k in range(13):
    testdfs.append(pd.DataFrame(columns=headers))
k=0
for i in range(len(tests)):
    print(tests[i])
    testdf = pd.read_csv(tests[i], names=headers, header=None)
    for j in range(13, 14):
        mask = testdf.customer_ID.isin(tdf[tdf['S_2']==j].customer_ID)
        testdfs[j-1] = pd.concat([testdfs[j-1], testdf[mask]], ignore_index=True)
        if testdfs[j-1]['customer_ID'].count() >= 260000:
            testdfs[j-1].iloc[:260000,].to_csv("t" + str(j-1) + "-" + str(k) + ".csv", index=False)
            testdfs[j-1] = testdfs[j-1].iloc[260000:,]
            k+=1            

testdfs[12].to_csv("t12-" + str(k) + ".csv", index=False)


In [5]:
predictions=pd.DataFrame()
for i in range(11, 12): #, 12):
    model = dtmodels[len(dtmodels)-1] #load_model(root+"/models/" + "x" + str(i) + "stackedreg")
    print("t" + str(i) + "dtcnt.csv")
    testdf = (pd.read_csv("t" + str(i) + "dtcnt.csv"))

    preds = predict_model(model, data=testdf, verbose=True)
    predictions = pd.concat([predictions, preds], ignore_index=True, axis=0)
    preds[['customer_ID', 'Label']].to_csv(root+"/test/t"+ str(i) + ".dtcnt" + ".preds", header=False, index=False)

#predictions.to_csv("test.preds", header=False, index=False)

t11dtcnt.csv


In [163]:
testdfs = []
i=0
for j in range(12):
    print("t" + str(j) + ".csv")
    testdfs.append(pd.read_csv("t" + str(j) + ".csv"))

    mindate = datetime.strptime('Apr 1 2018  1:33PM', '%b %d %Y %I:%M%p')
    maxdate = mindate + pd.offsets.DateOffset(months=j)
    dates=pd.DataFrame(pd.date_range(mindate, maxdate, freq='MS').tolist(), columns=['S_2_dt'])
    dates['yearmonth'] = dates['S_2_dt'].dt.strftime("%Y%m")

    testdfs[i]['S_2'] = pd.to_datetime(testdfs[i]['S_2'])
    testdfs[i]['yearmonth'] = testdfs[i]['S_2'].dt.strftime("%Y%m")
    testdfs[i][['sncnt', 'dncnt', 'rncnt', 'bncnt', 'pncnt']] = testdfs[i].apply(lambda x: (sum([(2^scols.index(col)) for col in scols if pd.isnull(x[col])]),
                sum([(2^dcols.index(col)) for col in dcols if pd.isnull(x[col])]),
                sum([(2^rcols.index(col)) for col in rcols if pd.isnull(x[col])]),
                sum([(2^bcols.index(col)) for col in bcols if pd.isnull(x[col])]),
                sum([(2^pcols.index(col)) for col in pcols if pd.isnull(x[col])])), axis=1, result_type='expand')

    dateshift = pd.DataFrame(testdfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'count'])).rename(columns={"min": "S_2"}).reset_index()
    dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
    dateshift['mondiff'] = dateshift['S_2'].dt.month - 4
    dateshift['mondiff'] = dateshift['mondiff'].apply(lambda x: 12+x if x <=0 else x)

    testdfs[i] = pd.merge(testdfs[i], dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(years=1) if x['S_2'].year==2019 and x['S_2'].month > 4 else x['S_2'], axis=1)
    testdfs[i]['yearmonthnew'] = testdfs[i]['S_2'].dt.strftime("%Y%m")
    testdfs[i]['ncount'] = testdfs[i].isnull().sum(axis=1)

    testdfs[i]=pd.merge(testdfs[i], pd.merge(pd.DataFrame(testdfs[i]['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), left_on=['customer_ID', 'yearmonthnew'], right_on=['customer_ID', 'yearmonth'], how='inner')
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2_dt'] if pd.isnull(x['S_2']) else x['S_2'], axis=1)
    testdfs[i].drop(columns=['S_2_dt', 'yearmonth_y'], inplace=True)
    testdfs[i] = testdfs[i].rename(columns={"yearmonth_x": "yearmonth"})

    testdfs[i]['mondiff'].fillna(0, inplace=True)
    testdfs[i]['day_of_month'] = testdfs[i]['S_2'].dt.strftime("%d")
    testdfs[i]['day_of_week'] = testdfs[i]['S_2'].dt.strftime("%u")
    testdfs[i]['day_of_year'] = testdfs[i]['S_2'].dt.strftime("%j")
    testdfs[i]['week_of_year'] = testdfs[i]['S_2'].dt.isocalendar().week
    testdfs[i].drop(columns=['mondiff'], inplace=True)
    testdfs[i].drop(columns=['S_2'], inplace=True)

#    df = testdfs[i].groupby("day_of_year").transform(lambda x: x.fillna(x.mean()))
#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col] = df[col]
    
#    df = testdfs[i].groupby("week_of_year").transform(lambda x: x.fillna(x.mean()))
#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col] = df[col]

#    df = testdfs[i].groupby("day_of_month").transform(lambda x: x.fillna(x.mean()))
#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col] = df[col]

#    df = testdfs[i].groupby("yearmonth").transform(lambda x: x.fillna(x.mean()))
#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col] = df[col]

#    df = testdfs[i].groupby("day_of_week").transform(lambda x: x.fillna(x.mean()))
#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col] = df[col]

    testdfs[i]=testdfs[i].set_index(['customer_ID', 'yearmonthnew']).unstack(1).sort_index(axis=1, level=1)
    testdfs[i] = testdfs[i].reset_index()
    testdfs[i] = pd.merge(testdfs[i], dateshift[['customer_ID', 'count']], on=['customer_ID'], how='inner')

#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col].fillna(testdfs[i][[column for column in testdfs[i].columns.tolist() if col[:len(col)-3] == column[:len(column)-3]]].mean(axis=1), inplace=True)

#    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
#        testdfs[i][col].fillna(testdfs[i][col].mean(), inplace=True)

    testdfs[i].drop(columns=[testdfs[i].columns.to_list()[1]], inplace=True)

    coldict={}
    for col in testdfs[i].columns:
        coldict[col] = ''.join(col)
    testdfs[i].rename(coldict, axis=1, inplace=True)

    testdfs[i].to_csv("t" + str(j) +"dtcnt.csv", index=False)
    i+=1

testdfs[0]

t0.csv
t1.csv
t2.csv
t3.csv
t4.csv
t5.csv
t6.csv
t7.csv
t8.csv
t9.csv
t10.csv
t11.csv


,customer_ID,B_1201804,B_10201804,B_11201804,B_12201804,B_13201804,B_14201804,B_15201804,B_16201804,B_17201804,...,day_of_week201804,day_of_year201804,dncnt201804,ncount201804,pncnt201804,rncnt201804,sncnt201804,week_of_year201804,yearmonth201804,count
0,0004ef34876259ea444bbe3c15031240a66cc73a7db0ef...,0.001889,0.297802,0.004482,0.415754,0.383141,0.012555,0.007515,0.085317,NaN,...,2,114,1062,24,0,37,0,17,201904,1
1,00085c285868ee0995a0d47eb096a229720abbf0fc8e3a...,0.005638,0.153685,0.007183,0.009568,0.007382,0.007703,0.000538,0.007939,NaN,...,2,100,1147,30,0,37,36,15,201904,1
2,002ea4ea3c03186c1d6a1c375843c85f17ff54a994f3e2...,0.158615,-0.000709,0.116474,0.013808,0.008102,0.035846,0.009827,0.004714,NaN,...,2,107,3083,61,3,61,0,16,201904,1
3,004900af6bce781c1538401a899a8a7ccccab64b95896b...,0.222732,0.298115,0.162289,0.005087,0.005549,0.028454,0.000634,0.006255,NaN,...,5,117,3168,64,3,61,0,17,201904,1
4,00639c0178fd720eb3e565558120a8b6bf040451cb7a88...,0.059552,0.133349,0.056337,0.112435,0.094728,0.128300,0.005942,0.591555,0.008546,...,2,114,1057,27,0,37,0,17,201904,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3899,ffb1ca6570e67b37ba9f0817d0686200e2d527c4590ef6...,0.061558,0.028597,0.040369,0.008627,0.009395,0.019425,0.009200,0.005948,NaN,...,7,105,2117,47,3,27,0,15,201904,1
3900,ffc3e0d0c0681e021cd305e583b69c4783a2d162faf18a...,0.192003,0.005672,0.152140,0.011209,0.012303,0.044810,0.002728,0.007848,NaN,...,4,109,3160,63,3,61,0,16,201904,1
3901,ffd14c0fe5226c6253141aa41a62a502a9ff8a9970b4e5...,0.009424,0.005620,0.004027,0.093704,NaN,0.025862,0.027792,0.008456,NaN,...,2,114,3089,62,3,61,0,17,201904,1
3902,ffd9110c2b6cc0afa50e3adb31e38386379419bacfd649...,0.180783,0.061928,0.142025,0.011307,NaN,0.046140,0.005505,0.009100,NaN,...,3,115,3168,65,3,61,0,17,201904,1


In [ ]:
testdfs = []
i=0
for j in range(41):
    print("t12-" + str(j) + ".csv")
    testdfs.append(pd.read_csv("t12-" + str(j) + ".csv"))

    mindate = datetime.strptime('Apr 1 2018  1:33PM', '%b %d %Y %I:%M%p')
    maxdate = mindate + pd.offsets.DateOffset(months=12)
    dates=pd.DataFrame(pd.date_range(mindate, maxdate, freq='MS').tolist(), columns=['S_2_dt'])
    dates['yearmonth'] = dates['S_2_dt'].dt.strftime("%Y%m")

    testdfs[i]['S_2'] = pd.to_datetime(testdfs[i]['S_2'])
    testdfs[i]['yearmonth'] = testdfs[i]['S_2'].dt.strftime("%Y%m")

    dateshift = pd.DataFrame(testdfs[i].groupby(['customer_ID'])['S_2'].agg(['min', 'count'])).rename(columns={"min": "S_2"}).reset_index()
    dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
    dateshift['mondiff'] = dateshift['S_2'].dt.month - 4
    dateshift['mondiff'] = dateshift['mondiff'].apply(lambda x: 12+x if x <0 else x)

    testdfs[i] = pd.merge(testdfs[i], dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2'] - pd.offsets.DateOffset(years=1) if x['S_2'].year==2019 and x['S_2'].month > 4 else x['S_2'], axis=1)
    testdfs[i]['yearmonthnew'] = testdfs[i]['S_2'].dt.strftime("%Y%m")
#    testdfs[i]['ncount'] = testdfs[i].isnull().sum(axis=1)

    testdfs[i]=pd.merge(testdfs[i], pd.merge(pd.DataFrame(testdfs[i]['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), left_on=['customer_ID', 'yearmonthnew'], right_on=['customer_ID', 'yearmonth'], how='inner')
    testdfs[i]['S_2'] = testdfs[i].apply(lambda x: x['S_2_dt'] if pd.isnull(x['S_2']) else x['S_2'], axis=1)
    testdfs[i].drop(columns=['S_2_dt', 'yearmonth_y'], inplace=True)
    testdfs[i] = testdfs[i].rename(columns={"yearmonth_x": "yearmonth"})

    testdfs[i]['mondiff'].fillna(0, inplace=True)
    testdfs[i]['day_of_month'] = testdfs[i]['S_2'].dt.strftime("%d")
    testdfs[i]['day_of_week'] = testdfs[i]['S_2'].dt.strftime("%u")
    testdfs[i]['day_of_year'] = testdfs[i]['S_2'].dt.strftime("%j")
    testdfs[i].drop(columns=['mondiff'], inplace=True)
    testdfs[i].drop(columns=['S_2'], inplace=True)

    testdfs[i]=testdfs[i].set_index(['customer_ID', 'yearmonthnew']).unstack(1).sort_index(axis=1, level=1)
    testdfs[i] = testdfs[i].reset_index()
    testdfs[i] = pd.merge(testdfs[i], dateshift[['customer_ID', 'count']], on=['customer_ID'], how='inner')

    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
        testdfs[i][col].fillna(testdfs[i][[column for column in testdfs[i].columns.tolist() if col[:len(col)-3] == column[:len(column)-3]]].mean(axis=1), inplace=True)

    for col in testdfs[i][testdfs[i].select_dtypes(include=['int64', 'float64']).columns].isna().any().index:
        testdfs[i][col].fillna(testdfs[i][col].mean(), inplace=True)

    testdfs[i].drop(columns=[testdfs[i].columns.to_list()[1]], inplace=True)

    coldict={}
    for col in testdfs[i].columns:
        coldict[col] = ''.join(col)
    testdfs[i].rename(coldict, axis=1, inplace=True)

    testdfs[i].to_csv("t12-" + str(j) +"dtcnt.csv", index=False)
#    os.remove(tests[i]
    i+=1

testdfs[0]

In [ ]:
#model = load_model("/amex/models/dt1Lclass")
#model = calibrated_dt
#model=stacked
#model=load_model("/amex/models/dtstackedreg")

useFlag = False
mindate = datetime.strptime('Apr 1 2018  1:33PM', '%b %d %Y %I:%M%p')

predictions=pd.DataFrame()
for i in range(104, len(tests)):
    print(tests[i].split("/")[5], end =" ")
    if (not useFlag):
        headers=pd.read_csv(root+"headers.csv")
        headers = headers.columns
        test = pd.read_csv(tests[i], names=headers, header=None)
        test['S_2'] = pd.to_datetime(test['S_2'])

        dateshift = pd.DataFrame(test.groupby(['customer_ID'])['S_2'].agg(['min', 'count'])).rename(columns={"min": "S_2"}).reset_index()
        dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
        dateshift['mondiff'] = ((dateshift['S_2'].dt.year * 12) + dateshift['S_2'].dt.month) - ((mindate.year*12) + mindate.month)

        test = pd.merge(test, dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
        test['S_2'] = test.apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
        test['yearmonth'] = test['S_2'].dt.strftime("%Y%m")

        test=pd.merge(test, pd.merge(pd.DataFrame(test['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
        test['S_2'] = test.apply(lambda x: x['S_2_dt'] if pd.isnull(x['S_2']) else x['S_2'], axis=1)

        test['day_of_month'] = test['S_2'].dt.strftime("%d")
        test['day_of_week'] = test['S_2'].dt.strftime("%u")
        test['day_of_year'] = test['S_2'].dt.strftime("%j")
        test.drop(columns=['S_2_dt'], inplace=True)
        test.drop(columns=['mondiff'], inplace=True)

        test = test.set_index(['customer_ID', 'yearmonth']).unstack(1).sort_index(axis=1, level=1)
        test = test.reset_index()
        test = pd.merge(test, dateshift[['customer_ID', 'count']], on=['customer_ID'], how='inner')

        test = test.merge(labels, on="customer_ID", how="left")
        test.drop(columns=[test.columns.to_list()[1]], inplace=True)

        coldict={}
        for col in test.columns:
            coldict[col] = ''.join(col)

        test.rename(coldict, axis=1, inplace=True)
        test['target'] = 0

#        test.to_csv(tests[i]+".csv", index=False, header=False)
    else:
        headers=pd.read_csv(root+"/trains/headers.csv")
        headers = headers.columns
        test = pd.read_csv(tests[i]+".csv", names=headers, header=None)

    for col in [col for col in test.columns if (col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']) or ('yearmonth' in col) or ('day_of_week' in col)]:
        test[col] = test[col].astype(str)
        test[col] = test[col].astype('category')

    test['customer_ID2'] = test['customer_ID']
    test.drop(columns=['customer_ID'], inplace=True)
    test.rename(columns = {"customer_ID2": "customer_ID"}, inplace=True)
    try:
        preds = predict_model(dtmodels[len(dtmodels)-1], data=test, verbose=True)
    except:
        preds = predict_model(dtmodels[0], data=test, verbose=True)
    predictions = pd.concat([predictions, preds], ignore_index=True, axis=0)
#    preds[['customer_ID', 'Label']].to_csv(tests[i]+"." + "".join(algos) + s
    preds[['customer_ID', 'Label']].to_csv(tests[i]+".dt2" + ".preds", header=False, index=False)

predictions.to_csv("test.preds", header=False, index=False)

In [ ]:
from datetime import datetime

mindate = datetime.strptime('Apr 1 2018  1:33PM', '%b %d %Y %I:%M%p')

test = pd.read_csv(tests[i], names=headers, header=None)
for col in [col for col in test.columns if col[:len(col)-6] in ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']]:
    test[col] = test[col].astype(str)
    test[col] = test[col].astype('category')

test['S_2'] = pd.to_datetime(test['S_2'])

dateshift = pd.DataFrame(test.groupby(['customer_ID'])['S_2'].min()).reset_index()
dateshift['S_2'] = pd.to_datetime(dateshift['S_2'])
dateshift['mondiff'] = ((dateshift['S_2'].dt.year * 12) + dateshift['S_2'].dt.month) - ((mindate.year*12) + mindate.month)

test = pd.merge(test, dateshift[['customer_ID', 'mondiff']], on=['customer_ID'], how='inner')
test['S_2'] = test.apply(lambda x: x['S_2'] - pd.offsets.DateOffset(months=x['mondiff']), axis=1)
test['yearmonth'] = test['S_2'].dt.strftime("%Y%m")

test=pd.merge(test, pd.merge(pd.DataFrame(test['customer_ID'].unique(), columns=['customer_ID']), dates, how='cross'), on=['customer_ID', 'yearmonth'], how='outer')
test['S_2'] = test.apply(lambda x: x['S_2_dt'] if pd.isnull(x['S_2']) else x['S_2'], axis=1)


In [ ]:
print((test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']['S_2'][0].year*12 + test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']['S_2'][0].month) - (mindate.year*12+mindate.month), (test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']['S_2'][0].year*100 + test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']['S_2'][0].month), (mindate.year*100+mindate.month))
test
test[test['customer_ID']=='0a31ea77ca60c94e754c884616778043f32f445ad2765f9564567030b425c185']

In [ ]:
def testcolfillna(col):
#    print(col, end="")
    try:
        test[col] = test[col[:len(col)-6]+str(int(col[len(col)-6:len(col)-2])+1) + str(col[len(col)-2:len(col)])]
    except:
        print(col)
    return

def testfillna(col):
#    print(".", end="")
    test[col].fillna(test[[column for column in test.columns.tolist() if column[:len(column)-3] in col]].mean(axis=1), inplace=True)

#    with Pool(12) as pool:
#        pool.map(testcolfillna, [col for col in test.columns[test.isna().all()]])

#    with Pool(12) as pool:
#        pool.map(testfillna, [col for col in test.columns[test.isna().any()]])

#    for col in test.columns[test.isna().all()]:
#        test[col] = test[col[:len(col)-6]+str(int(col[len(col)-6:len(col)-2])+1) + str(col[len(col)-2:len(col)])]



In [ ]:
traindf = pd.read_csv(root+"/x12-0.csv")
traindf